In [164]:
"""
Testing strategies
"""
import random
import statistics


def get_random(hand, other):
    # Select randomly from possible values 
    guess = random.sample(list(other), 12)

    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct


def get_below_max(hand, other):
    # Max strategy 
    max_val = max(hand)
    below = [val for val in other if val < max_val]

    # Select randomly from possible values 
    guess = random.sample(below, 12)

    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct


def get_most_in_window(hand, other, window=12, highest=52):
    # Just for easier debugging
    hand.sort()
    
    # In YOUR HAND finding the window with the highest number of cards 
    min_window = 0
    num_cards_in_window = 0
    for min_card in hand:
        # Values in window that are ABOVE the min_window but BELOW the highest value (ie 52) 
        vals_in_window = [i for i in range(min_card + 1, min_card + window + 1) if i <= highest]
        
        # This means that min_window > 40 (because then the numbers to check for go above 53 which is out of range).
        if len(vals_in_window) < window:
            looped_window = [i for i in range(1, (min_card + window + 1) - highest)]
            vals_in_window = vals_in_window + looped_window
        
        cards_in_window = set(hand).intersection(set(vals_in_window))
        
        if len(cards_in_window) > num_cards_in_window:
            num_cards_in_window = len(cards_in_window)
            min_window = min_card
            
    # Your PARTNER using that min_window to get their guesses
    # Now use this min_window to guess cards from other 
    guess = [val for val in other if min_window < val]
    
    # If there are not enough guessable cards above the min_window loop back to select from the bottom
    if len(guess) < window:
        max_window = window - len(guess) 
        other.sort()
        guess = guess + other[:max_window]
    else:
        guess = guess[:window]
        
    assert len(guess) == window
    
    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct    



In [167]:
random_score = []
max_score = []
window_score = []
num_runs = 10000
for exp in range(0,num_runs):
    deck = [x for x in range(1,53)]
    hand = random.sample(deck, 13)
    part = random.sample(list(set(deck) - set(hand)), 13)
    other = list(set(deck) - set(part))
    
    r_guess, r_num_correct = get_random(hand, other)
    random_score.append(r_num_correct)
    m_guess, m_num_correct = get_below_max(hand, other)
    max_score.append(m_num_correct)
    w_guess, w_num_correct = get_most_in_window(hand, other)
    window_score.append(w_num_correct)

print(f"Over {num_runs} runs")
print("Random selection -- Avg R1 Score: ", statistics.mean(random_score), "SD: ", round(statistics.stdev(random_score), 3))
print("Max bound selection -- Avg R1 Score: ", statistics.mean(max_score), "SD: ", round(statistics.stdev(max_score), 3))
print("Most cards in window selection -- Avg R1 Score:", statistics.mean(window_score), "SD: ", round(statistics.stdev(window_score), 3))

Over 10000 runs
Random selection -- Avg R1 Score:  4.0014 SD:  1.368
Max bound selection -- Avg R1 Score:  3.9839 SD:  1.388
Most cards in window selection -- Avg R1 Score: 5.378 SD:  0.951
